In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_targets_scored.csv


In [2]:
# Importing the necessary Libraries
import numpy as np
import pandas as pd
import os 
import sys
import matplotlib.pyplot as plt
from pandas import plotting
from sklearn import preprocessing
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
from sklearn.metrics import log_loss
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import RobustScaler
import pandas_profiling
from catboost import CatBoostClassifier
from catboost import cv
from catboost import Pool

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
trainTargets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
trainTargetsNonScored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
combinedTrain = train.merge(trainTargets)

In [4]:
featureCols = train.columns
labelCols = trainTargets.drop('sig_id',axis=1).columns

In [5]:
# shuffle the dataset
combinedTrain = combinedTrain.sample(frac=1, random_state= 2)
# delete the index
combinedTrain.reset_index(drop=True, inplace=True)
# show the head of combined df
combinedTrain.head()
# reconstruct the dataframe 
train = combinedTrain[featureCols]
trainTargets = combinedTrain[labelCols]

In [6]:
# check the cleanliness of the data 
train.isnull().sum().sum()

0

In [7]:
# One hot Encoding
forEncoding = ['cp_time', 'cp_type', 'cp_dose']
train = pd.concat([train,(pd.get_dummies(train[forEncoding].astype(str),dummy_na=False, dtype=np.uint8,prefix="feature"))],axis=1)
test = pd.concat([test,(pd.get_dummies(test[forEncoding].astype(str),dummy_na=False, dtype=np.uint8,prefix="feature"))],axis=1)

In [8]:
# Get columns of features and labels
labelCols = [col for col in trainTargets.columns if col != 'sig_id']
featureCols = [col for col in train.columns if col not in ['sig_id', 'cp_time', 'cp_type', 'cp_dose']]
print('Number of different labels:', len(labelCols))
print('Number of features:', len(featureCols))

Number of different labels: 206
Number of features: 879


In [9]:
# Separate the Dataset
X = train[featureCols]
X_test = test[featureCols]
y = trainTargets

In [10]:
cat_features= ['feature_24',
 'feature_48',
 'feature_72',
 'feature_ctl_vehicle',
 'feature_trt_cp',
 'feature_D1',
 'feature_D2']
nonCat = [col for col in X.columns if col not in cat_features]

In [11]:
# Standardize the data using a robust scalar
rsc = RobustScaler()
X[nonCat] = rsc.fit_transform(X[nonCat])
X_test[nonCat] = rsc.transform(X_test[nonCat])
X = pd.DataFrame(X)
X_test = pd.DataFrame(X_test)

In [12]:
X

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,...,c-97,c-98,c-99,feature_24,feature_48,feature_72,feature_ctl_vehicle,feature_trt_cp,feature_D1,feature_D2
0,1.441616,1.165057,-0.423900,-0.647555,0.962939,-1.147126,0.487031,1.510978,0.954850,-0.855455,...,0.487989,0.873159,0.522890,0,1,0,1,0,0,1
1,0.985457,0.119655,-0.332494,-0.209244,0.140874,0.400440,-0.037697,-0.111411,0.306532,-0.416881,...,-0.730693,-2.281105,-0.116519,0,1,0,0,1,0,1
2,-0.070136,0.778742,-0.139152,-0.081245,0.251415,0.450681,0.582178,0.582451,-0.720992,-0.203077,...,-0.555986,-0.625501,-0.789876,0,0,1,0,1,0,1
3,0.025581,0.562120,-1.713028,0.349898,-0.186265,-0.264279,-0.205403,-0.404237,-0.112169,1.591295,...,-0.018463,-0.484593,0.582897,0,0,1,0,1,0,1
4,-0.529298,0.413354,0.082601,-0.154036,-0.138524,-0.627825,-0.851192,0.432303,0.830710,1.336090,...,0.472503,-0.308670,0.691528,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,0.039599,-0.379581,-0.470375,0.838041,0.680337,0.407451,0.121794,-0.011312,-0.224244,-0.054207,...,0.020449,0.185506,0.698417,0,0,1,0,1,1,0
23810,1.817076,-0.215068,-0.355731,1.783869,0.762897,-0.246304,1.643467,-0.246832,-4.740315,-0.695999,...,-0.753028,-4.519511,-0.493835,1,0,0,0,1,0,1
23811,0.161448,-0.017508,1.232487,-0.760548,1.058421,-0.783849,-0.284904,0.790118,-0.540956,1.510953,...,-0.033651,-1.175661,-0.414957,1,0,0,0,1,1,0
23812,0.453105,0.329336,0.574761,-0.661665,-0.144291,-0.490226,0.588437,-0.117920,1.612782,0.119521,...,0.695057,0.845072,0.103140,1,0,0,0,1,0,1


In [13]:
# Convert to numpy array
X = X.iloc[:].to_numpy()
X_test = X_test.iloc[:].to_numpy()
y = y.iloc[:].to_numpy()

In [14]:
# Let's look at the data size
print(X.shape)
print(X_test.shape)
print(y.shape)

(23814, 879)
(3982, 879)
(23814, 206)


In [15]:
# Importing Libraries
import numpy as np 
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputClassifier

import warnings
warnings.filterwarnings('ignore')

In [16]:
SEED = 42
NFOLDS = 5
DATA_DIR = '/kaggle/input/lish-moa/'
np.random.seed(SEED)

In [17]:
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')

test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

# drop id col
X = train.iloc[:,1:].to_numpy()
X_test = test.iloc[:,1:].to_numpy()
y = targets.iloc[:,1:].to_numpy() 

In [18]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

clf = Pipeline([('encode', CountEncoder(cols=[0, 2])),
                ('classify', classifier)
               ])

In [19]:
params = {'classify__estimator__colsample_bytree': 0.6522,
          'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0503,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
          'classify__estimator__subsample': 0.8639
         }

_ = clf.set_params(**params)

In [20]:
oof_preds = np.zeros(y.shape)
test_preds = np.zeros((test.shape[0], y.shape[1]))
oof_losses = []
kf = KFold(n_splits=NFOLDS)
for fn, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print('Starting fold: ', fn)
    X_train, X_val = X[trn_idx], X[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]
    
    # drop where cp_type==ctl_vehicle (baseline)
    ctl_mask = X_train[:,0]=='ctl_vehicle'
    X_train = X_train[~ctl_mask,:]
    y_train = y_train[~ctl_mask]
    
    clf.fit(X_train, y_train)
    val_preds = clf.predict_proba(X_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    oof_preds[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    oof_losses.append(loss)
    preds = clf.predict_proba(X_test)
    preds = np.array(preds)[:,:,1].T # take the positive class
    test_preds += preds / NFOLDS
    
print(oof_losses)
print('Mean OOF loss across folds', np.mean(oof_losses))
print('STD OOF loss across folds', np.std(oof_losses))

Starting fold:  0
Starting fold:  1
Starting fold:  2
Starting fold:  3
Starting fold:  4
[0.0169781773377249, 0.01704491710861325, 0.016865153552168475, 0.01700900926983899, 0.01717882474706338]
Mean OOF loss across folds 0.017015216403081797
STD OOF loss across folds 0.00010156682747757948


In [21]:
# CatBoost
train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
trainTargets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
trainTargetsNonScored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
combinedTrain = train.merge(trainTargets)
featureCols = train.columns
labelCols = trainTargets.drop('sig_id',axis=1).columns

In [22]:
# shuffle the dataset
combinedTrain = combinedTrain.sample(frac=1, random_state= 2)
# delete the index
combinedTrain.reset_index(drop=True, inplace=True)
# show the head of combined df
combinedTrain.head()
# reconstruct the dataframe 
train = combinedTrain[featureCols]
trainTargets = combinedTrain[labelCols]

In [23]:
# Get columns of features and labels
labelCols = [col for col in trainTargets.columns if col != 'sig_id']
featureCols = [col for col in train.columns if col !='sig_id']
print('Number of different labels:', len(labelCols))
print('Number of features:', len(featureCols))

Number of different labels: 206
Number of features: 875


In [24]:
lEncode = preprocessing.LabelEncoder()
forEncoding = ['cp_time','cp_type','cp_dose']
# for train

for i in forEncoding:
    lEncode.fit(train[i])
    x = lEncode.transform(train[i])
    train[i] = x

# for test
for i in forEncoding:
    lEncode.fit(test[i])
    x = lEncode.transform(test[i])
    test[i] = x

In [25]:
# Separate the Dataset
X = train[featureCols]
X_test = test[featureCols]
y = trainTargets

In [26]:
cat_features= ['cp_time', 'cp_type', 'cp_dose']
nonCat = [col for col in X.columns if col not in cat_features]

In [27]:
# Standardize the data using a robust scalar
rsc = RobustScaler()
X[nonCat] = rsc.fit_transform(X[nonCat])
X_test[nonCat] = rsc.transform(X_test[nonCat])
X = pd.DataFrame(X)
X_test = pd.DataFrame(X_test)

In [28]:
# Convert to numpy array
X = X.iloc[:].to_numpy()
X_test = X_test.iloc[:].to_numpy()
y = y.iloc[:].to_numpy()

In [29]:
y.shape

(23814, 206)

In [30]:
cat_features= ['cp_time', 'cp_type', 'cp_dose']

# second model - Catboost
params = {'loss_function':'MultiClass',
          #'eval_metric':'los',
          #'cat_features': cat_features,
          'task_type': 'GPU',
          'border_count': 32,
          'verbose': 200,
          'random_seed': 1,
          #'learning_rate' : 0.1,
          #'random_strength' : 0.1,
          #'depth' : 8,
          'early_stopping_rounds' : 200,
          #'leaf_estimation_method' : 'Newton'
         }
cbc = CatBoostClassifier(**params)
classifier = MultiOutputClassifier(cbc)

In [31]:
classifier.fit(X, y)
preds = classifier.predict_proba(X_test)
preds = np.array(preds)[:,:,1].T
test_preds = (test_preds/2)  + (preds/2)

Learning rate set to 0.123925
0:	learn: 0.5788456	total: 23.3ms	remaining: 23.3s
200:	learn: 0.0015470	total: 1.78s	remaining: 7.09s
400:	learn: 0.0004285	total: 3.42s	remaining: 5.11s
600:	learn: 0.0002200	total: 5.53s	remaining: 3.67s
800:	learn: 0.0001337	total: 7.75s	remaining: 1.93s
999:	learn: 0.0000958	total: 9.44s	remaining: 0us
Learning rate set to 0.123925
0:	learn: 0.5785123	total: 26.3ms	remaining: 26.3s
200:	learn: 0.0020814	total: 1.8s	remaining: 7.16s
400:	learn: 0.0005851	total: 3.51s	remaining: 5.24s
600:	learn: 0.0002760	total: 5.67s	remaining: 3.77s
800:	learn: 0.0001689	total: 7.43s	remaining: 1.84s
999:	learn: 0.0001175	total: 9.17s	remaining: 0us
Learning rate set to 0.123925
0:	learn: 0.5788959	total: 11.5ms	remaining: 11.5s
200:	learn: 0.0032449	total: 1.68s	remaining: 6.69s
400:	learn: 0.0010472	total: 3.34s	remaining: 4.99s
600:	learn: 0.0004746	total: 6.02s	remaining: 4s
800:	learn: 0.0002863	total: 7.67s	remaining: 1.91s
999:	learn: 0.0001989	total: 9.41s	re

In [32]:
test_preds

array([[0.00103385, 0.00110314, 0.00107265, ..., 0.0011307 , 0.00095416,
        0.00108185],
       [0.00102359, 0.00104697, 0.00104646, ..., 0.0010314 , 0.00128173,
        0.00106562],
       [0.0010237 , 0.00102994, 0.0011167 , ..., 0.00107976, 0.00116307,
        0.00105411],
       ...,
       [0.00102828, 0.00103868, 0.0010343 , ..., 0.00104377, 0.00111629,
        0.0010337 ],
       [0.00102361, 0.00103303, 0.0010542 , ..., 0.00109358, 0.00113967,
        0.00110161],
       [0.00102652, 0.0010828 , 0.00105639, ..., 0.00112066, 0.00085079,
        0.00107954]])

In [33]:
# set control test preds to 0
mask = test['cp_type']=='ctl_vehicle' # wherever the cp_type = 'ctl_vehicle' 

test_preds[mask] = 0

In [34]:
sub = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
sub.iloc[:,1:] = test_preds

In [35]:
sub = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
sub.iloc[:,1:] = test_preds

In [36]:
sub.to_csv('submission.csv', index=False)